In [1]:
import os
os.environ['USE_PYGEOS'] = '0'

from functions import prepare_folders, download_images_for_points, get_gvi_per_buffer, select_points_on_road_network, get_features_on_points, get_road_network_with_points, select_points_within_buffers
import geopandas as gpd
from datetime import timedelta
from time import time

In [2]:
file = "Test_multiple_home_locations.gpkg" # File with address points
layer = "Test_multiple_home_locations"
buffer_distance = 500 # buffer distance in meters
access_token = "MLY|6267906093323631|fba37c53726a386c951323ee5b9874bf" # Access token for mapillary

In [3]:
prepare_folders()

In [4]:
# Read the file as a GeoDataFrame
gdf_points = gpd.read_file(file, layer=layer) # The crs must use meters
gdf_points['buffer'] = gdf_points['geometry'].buffer(buffer_distance)

In [5]:
# Reproject the GeoDataFrame to WGS84 (EPSG:4326)
gdf_points.to_crs(epsg=4326, inplace=True)

# Reproject the 'buffer' column to EPSG 4326 
gdf_points['buffer'] = gdf_points['buffer'].to_crs(epsg=4326)

In [6]:
road = get_road_network_with_points(gdf_points['buffer'])

In [7]:
road_points = select_points_on_road_network(road)

In [8]:
road_points

,geometry
0,POINT (551927.657 5922905.399)
1,POINT (551899.402 5922946.125)
2,POINT (551946.604 5922910.291)
3,POINT (551895.939 5922950.201)
4,POINT (551880.236 5922887.434)
...,...
3466,POINT (554728.995 5923022.647)
3467,POINT (554723.874 5923072.289)
3468,POINT (554326.754 5921770.777)
3469,POINT (551893.606 5921475.896)


In [9]:
points = select_points_within_buffers(gdf_points, road_points)

In [10]:
points

,geometry
0,POINT (-2.21852 53.44321)
1,POINT (-2.21855 53.44276)
2,POINT (-2.21854 53.44264)
3,POINT (-2.20678 53.43799)
4,POINT (-2.20751 53.43808)
...,...
701,POINT (-2.17676 53.45339)
702,POINT (-2.17667 53.45297)
703,POINT (-2.17592 53.45301)
704,POINT (-2.17583 53.45337)


In [11]:
# Get features for each point
features = get_features_on_points(points, access_token)

file_path = os.path.join("results", "points.gpkg")
features.to_file(file_path, driver="GPKG")

# Get the initial time
start_time = time()

# The number of workers must be adapted depending on the computer
gvi_per_point = download_images_for_points(features, access_token, 2)
# Get the final time
end_time = time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Format the elapsed time as "hh:mm:ss"
formatted_time = str(timedelta(seconds=elapsed_time))

print(f"Running time: {formatted_time}")

Processing images (Process 2): 100%|██████████| 88/88 [05:15<00:00,  3.58s/it]


Running time: 0:05:24.982689


In [12]:
gvi_buffer, points = get_gvi_per_buffer(gdf_points, gvi_per_point)

In [13]:
display(gvi_buffer)

,geometry,GVI,Point_Count
0,POINT (-2.17238 53.43243),0.074332,4
1,POINT (-2.21250 53.44085),0.255787,47
2,POINT (-2.17267 53.45455),NaN,0


In [14]:
display(points)

# index has the UID of the point (or buffer)
# geometry_left has the coordinates of the sample point that is within the buffer
# GVI has the GVI value, NaN in case the image is missing
# is_panoramic indicates if the image is panoramic, None in case it is missing
# missing indicates if the images was usable (road centers were identified) or not

,index_right,level_1,geometry_left,GVI,is_panoramic,missing
0,0,0,POINT (-2.17795 53.42943),0.041184,False,False
1,0,1,POINT (-2.16809 53.43352),NaN,None,True
2,0,2,POINT (-2.16715 53.43379),NaN,None,True
3,0,3,POINT (-2.16857 53.43363),NaN,None,True
4,0,4,POINT (-2.16929 53.43377),NaN,None,True
...,...,...,...,...,...,...
701,2,239,POINT (-2.17375 53.45139),NaN,None,True
702,2,240,POINT (-2.17350 53.45361),NaN,None,True
703,2,241,POINT (-2.17504 53.45389),NaN,None,True
704,2,242,POINT (-2.17256 53.45395),NaN,None,True


In [15]:
file_path = os.path.join("results", "GVI-points.gpkg")
points.to_file(file_path, driver="GPKG")

file_path = os.path.join("results", "GVI-buffer.gpkg")
gvi_buffer.to_file(file_path, driver="GPKG")